In [ ]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install fastText
import fasttext.util
fasttext.util.download_model('ru', if_exists='ignore')  
model = fasttext.load_model('cc.en.300.bin')

In [ ]:
! git clone https://github.com/facebookresearch/fastText.git
! cd fastText
! python setup.py install

fatal: destination path 'fastText' already exists and is not an empty directory.
python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [ ]:
! git clone https://github.com/facebookresearch/fastText.git
! cd fastText
! sudo python setup.py install

fatal: destination path 'fastText' already exists and is not an empty directory.
python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [ ]:
import fasttext

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
!pip install pymorphy2
from sklearn.linear_model import LogisticRegression 

     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 8.2 MB 38.2 MB/s 


In [ ]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [ ]:
train['toxic'] = train.toxic.astype(int)
y_train = train['toxic']
y_train

0        0
1        0
2        0
3        0
4        0
        ..
10804    0
10805    1
10806    1
10807    0
10808    0
Name: toxic, Length: 10809, dtype: int64

In [ ]:
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

m = MorphAnalyzer()
regex = re.compile("[а-яa-zё]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [ ]:
from tqdm import tqdm

lemmas = list(tqdm(map(clean_text, train['comment']), total=len(train)))
    
train['lemmas'] = lemmas
train.sample(5)

100%|██████████| 10809/10809 [00:50<00:00, 215.02it/s]


,comment,toxic,lemmas
9512,Я от 1410 ПГ в ультразвуковой ванночке неделю ...,0,ультразвуковой ванночка неделя отмачивать печа...
947,Меня на ностальгию пробило. До 18 лет жила там...,0,ностальгия пробить жить лететь гость родитель ...
5091,как лебиртарианцы могут быть против ограничени...,0,лебиртарианец мочь против ограничение свобода ...
8114,Мамке твоей на вайбер\n,1,мамка твой вайбер
6443,да хз) может раньше была такая тенденция) я ли...,0,мочь ранний тенденция лично совместный бюджет


In [ ]:
lemmas_test = list(tqdm(map(clean_text, test['comment']), total=len(test)))
    
test['lemmas'] = lemmas_test

100%|██████████| 3603/3603 [00:17<00:00, 211.67it/s]


In [ ]:
vec = TfidfVectorizer(ngram_range=(1, 2)) # строим BoW для слов
bow = vec.fit_transform(train['lemmas'])

clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)

pred = clf.predict(vec.transform(test['lemmas']))

In [ ]:
baseline_5 = [[i, pred[i]] for i in range(len(test))]
baseline_5 = pd.DataFrame(baseline_5, columns=['comment_id', 'toxic'])

baseline_5.to_csv('baseline_5.csv', index=False)

In [ ]:
baseline_5.head(15)

,comment_id,toxic
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,1
9,9,0


In [ ]:
train

,comment,toxic,lemmas
0,Преступление и наказание\n,0,преступление наказание
1,"И именно эти неработающие весы показывают, что...",0,именно неработающий весы показывать работать
2,"В Японии такие панельки, ебанько.\n",0,япония панелька ебанько
3,Еще у нас выявляют трещины с помощью белой кра...,0,выявлять трещина помощь белый краска магнитная...
4,"Дочитал до поезда в Норильск , дальше не стал\n",0,дочитать поезд норильск далёкий стать
...,...,...,...
10804,А у мамы в группе до самого выпуска из сада та...,0,мама группа выпуск просяк случаться разный реб...
10805,Сука тупой дегенарт. Вот на эти видео. Съеби у...,1,сука тупой дегенарта видео съести свой старый ...
10806,"В племенах украинцев, особенно западных, с дет...",1,племя украинец особенно западный детство приви...
10807,"Пост про жадность, о том, как человек оплативш...",0,пост жадность человек оплатить предоставить ха...


In [ ]:
test

,comment_id,comment,lemmas
0,0,"В данном случае не пионервожатая, а техничка, ...",данный случай пионервожатый техничка полагать ...
1,1,В данном посте рассматривается обычный ленточн...,данный пост рассматриваться обычный ленточный ...
2,2,"Как я понимаю, у поверхностей отражаемость пог...",понимать поверхность отражаемость поглощаемост...
3,3,Австралийские пауки съедят все живое на планете)),австралийский паук съесть живой планета
4,4,У нас раньше Амур пиво целые бутылки принимал ...,ранний амур пиво целый бутылка принимать свой ...
...,...,...,...
3598,3598,"Я всё жду, когда эта херня закончится\n",ждать херня закончиться
3599,3599,Откуда такой акцент на Москве? У нас между про...,откуда акцент москва прочить многонациональный...
3600,3600,"Кого угодно, но не этих двух отбросов: свинью ...",угодный отброс свинья недоразвитый даун
3601,3601,А как же радиоуправляемые машинки в клубах?\n,радиоуправляемый машинка клуб


In [ ]:
with open('train_ft.txt', 'w') as f:
    for pair in list(zip(train['lemmas'], train['toxic'])):
        text, label = pair
        f.write(f'__label__{label} {text.lower()}\n')

In [ ]:
pred = classifier.predict(list(test['lemmas']))[0]
pred = [int(label[0][-1]) for label in pred]

In [ ]:
baseline_6 = [[i, pred[i]] for i in range(len(test))]
baseline_6 = pd.DataFrame(baseline_6, columns=['comment_id', 'toxic'])

baseline_6.to_csv('baseline_6.csv', index=False)

In [ ]:
baseline_6.head(15)

,comment_id,toxic
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,1
9,9,0
